In [2]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [3]:
from pathlib import Path
import re
from collections import OrderedDict
from tqdm.notebook import tqdm
import pprint
import json
import abc

## LayoutオブジェクトからLTTextBoxのリストを取得する関数

抜き出すのは，textデータを前提とするのでこの関数が必要

In [4]:
def find_textboxes_recursively(layout):
    """
    再帰的にテキストボックス（LTTextBox）を探して、テキストボックスのリストを取得する。
    """
    # LTTextBoxを継承するオブジェクトの場合は1要素のリストを返す。
    if isinstance(layout, LTTextBox):
        text_boxes = [layout]
        return text_boxes  # 返すのはリスト

    # LTContainerを継承するオブジェクトは子要素を含むので、再帰的に探す。
    if isinstance(layout, LTContainer):
        text_boxes = []
        for child in layout:
            text_boxes.extend(find_textboxes_recursively(child))  # 再帰的にリストをextend
            
        return text_boxes

    return []  # 何も取得できなかった場合は空リストを返す

## ソート用の関数 

textboxのソートは，1段組みと2段組みで異なる

### 二段組用のソート 

In [5]:
class SortTextbox2Column():
    """
    2段組み用，始めのソートは左側と右側
    """
    def __init__(self, layout_x0, layout_x1):
        self.half_x = (layout_x0 + layout_x1)/2
    
    def __call__(self, text_box):
        if text_box.x0 < self.half_x:
            left_or_right = -1  # it mean left
            
        else:
            left_or_right = 1  # it mean right
            
        return (left_or_right, -text_box.y1)

### 1段組み用のソート 

In [6]:
class SortTextbox():
    """
    textboxの左下の座標でソート
    """
    def __init__(self,*args):
        """
        2段組み用のソートクラスとの対応のため
        """
        pass
    def __call__(self, text_box):
        return (-text_boxt.y1, text_box.x0)

## 論文データのベースクラス 

pdfデータをパースして保存するときと，呼び出すときに利用する？

In [16]:
class PaperBase(metaclass=abc.ABCMeta):
    """
    論文のデータクラスとPaser用のストラテジーを一つにしたもの.正直一つにする意味はない.
    ただ変更するクラスをまとめただけ
    
    """
    @abc.abstractmethod
    def toDict(self):
        pass
    
    @classmethod
    def parse_by_textboxes(cls, text_boxes, parse_info):
        """
        text_boxesからパースする
        """
        paper_title, parse_text_dict = cls.str_from_textboxes(text_boxes, parse_info)  # スタティクメソッド
        paper = cls.parse_by_text_dict(paper_title=paper_title, 
                                       parse_text_dict=parse_text_dict, 
                                       parse_info=parse_info)  # クラスメソッド
        
        return paper
    
    @classmethod
    def parse_by_text_dict(cls, paper_title, parse_text_dict, parse_info):
        raise NotImplementedError("Implement parse_by_text_dict")
        
        
    @classmethod
    def parse_by_dict(cls, content):
        raise NotImplementedError("Implement parse_by_content")
        
    @staticmethod
    def str_from_textboxes(text_boxes, parse_info):
        """
        共通するテキスト取得プログラム
        """
        parse_text_flag = False  # このフラッグがTrueである部分を保存する        
                
        patterns_keys = parse_info["start_patterns"].keys()  # キーのリスト(のようなもの)
        patterns_key_iter = iter(patterns_keys)  # 長さの違うfor文内で回すので，キーをイテレーター化
        pattern_key = next(patterns_key_iter)  # 最初のキーを取得
        
        parse_text_dict = {i:"" for i in patterns_keys}
        
        for i,box in enumerate(text_boxes):
            text = box.get_text().strip()  # 末尾の文字を削除
            if i == parse_info["title_position_number"]:
                paper_title = text

            if parse_text_flag:  # flagがTrueのうちは，parse_textにtextを加え続ける
                parse_text_dict[pattern_key] += text

            if parse_info["start_patterns"][pattern_key].search(text):  # マッチしたらフラッグをTrueに
                parse_text_flag = True
            
            if parse_info["end_patterns"][pattern_key] is not None:  # Noneだったら，最後までflagはTrue
                if parse_info["end_patterns"][pattern_key].search(text):
                    try:
                        parse_text_flag = False
                        pattern_key = next(patterns_key_iter)  # end_patternがマッチしたらpatterns_key_iterをイテレーション
                    except StopIteration:
                        break  # 次のpattern_keyがなくなってStopIterationエラーが出たら終了
        
        return paper_title, parse_text_dict

### セーブデータ保存用の Paperクラス

In [17]:
class PaperForSave(PaperBase):
    """
    論文をテキストデータとして，保存するための論文データクラス
    """
    def __init__(self, 
                 conf_name=None, 
                 pdf_name=None, 
                 paper_title=None, 
                 pdf_content=None,
                 
                ):
        """
        一つのデータで
        Parameters
        ----------
        conf_name: str
            学会や論文集を表す文字列
        pdf_name: str
            対応するpdfファイルの名前を表す文字列
        paper_title: str
            論文のタイトル
        pdf_content: dict
            保存するテキストのdictionaly
        """
        self.conf_name = conf_name
        self.pdf_name = pdf_name
        self.paper_title = paper_title
        self.pdf_content = pdf_content
        
    def toDict(self):
        out_dict = {"pdf_name": self.pdf_name,
                    "paper_title": self.paper_title,
                    "content":self.pdf_content
                   }
        return out_dict
    
    @classmethod
    def parse_by_text_dict(cls, paper_title, parse_text_dict, parse_info):
        """
        Parameters
        ----------
        text_boxes: list of textbox
            pdfをパースしたときに得られるtextboxのリスト
        start_patterns
        """
        #from IPython.core.debugger import Pdb; Pdb().set_trace()  # PaperForSave
        paper_conf_name = parse_info["conf_name"]
        paper_pdf_name = parse_info["pdf_name"]
        
        # Paperへのデータの付与
        paper = cls(conf_name=paper_conf_name,
                    paper_title=paper_title,
                    pdf_name=paper_pdf_name,
                    pdf_content=parse_text_dict
                   )

        return paper


### カウント用のPaperクラス 

Paperクラスの拡張は以下のように行う，初期化メソッドはデータをアトリビュートとして保持するように実装．`toDict`と`parse_by_textboxes`,`parse_by_contents`は適宜実装する．その際，Parserクラスの`parse_info`と対応するように実装する．

In [22]:
class PaperForCount(PaperBase):
    def __init__(self, pdf_name=None, count_patterns=[], paper_title=None):
        """
        countersは保存する文字列あるいはパターンのリスト
        """
        self.pdf_name = pdf_name
        self.paper_title = paper_title
        self.counters = OrderedDict()
        for i in count_patterns:
            self.counters[i] = 0  # パターンオブジェクトはhashableでキーにできる．まず，0に初期化
    
    def toDict(self):
        counters = {i.pattern:self.counters[i] for i in self.counters.keys()}  # キーを文字列へ
        
        out_dict = {"pdf_name":self.pdf_name,
                    "paper_title":self.paper_title,
                    "counters":counters
                   }
        return out_dict
    
    @classmethod
    def parse_by_text_dict(cls, paper_title, parse_text_dict, parse_info):
        """
        Parameters
        ----------
        text_boxes: list of textbox
            pdfをパースしたときに得られるtextboxのリスト
        parse_info: dict 
            パースの時に必要な情報
        """
        #from IPython.core.debugger import Pdb; Pdb().set_trace()  # PaperForCount
        
        paper_pdf_name = parse_info["pdf_name"]
                
        # 以下Paperへのデータの付与
        count_patterns = parse_info["count_patterns"]
        
        paper = cls(pdf_name=paper_pdf_name,
                    paper_title=paper_title,
                    count_patterns=count_patterns
                   )
        
        for pattern in count_patterns:
            for text in parse_text_dict.values():
                m = pattern.findall(text)
                paper.counters[pattern] += len(m)
                
        return paper
    
    @classmethod
    def parse_by_dict(cls, paper_dict, parse_info):
        #from IPython.core.debugger import Pdb; Pdb().set_trace()  # PaperForCount
        paper_title = paper_dict["paper_title"]
        paper_pdf_name = paper_dict["pdf_name"]
        parse_text_dict = paper_dict["contents"]
        
        count_patterns = parse_infonfo["count_patterns"]
        
        paper = cls(pdf_name=paper_pdf_name,
                    paper_title=paper_title,
                    count_patterns=count_patterns
                   )
        
        for pattern in count_patterns:
            for text in parse_text_dict.values():
                m = pattern.findall(text)
                paper.counters[pattern] += len(m)
                
        return paper
    
    def is_counted(self):
        # 一つも含まれていないとき
        if set(self.counters.values()) == {0}:
            return False
        else:
            return True
        
    def __repr__(self):
        str_pdf_name = str(self.pdf_name)
        str_paper_title = str(self.paper_title)
        str_counters = str(self.counters)
        return str_pdf_name+"\n"+str_paper_title+"\n"+str_counters

## あるpdfファイルをパースし，パースした内容をPaperオブジェクトで返すオブジェクト

In [23]:
class PdfParser():
    def __init__(self, 
                 conference_name,
                 start_patterns={"all":re.compile(".*")},
                 end_patterns={"all":None},
                 title_position_number=2,
                 parse_page_numbers=[0],
                 column_number=2,
                 paper_data_class=PaperForSave()
                ):
        """
        Parameters
        ----------
        conference_name: str
            学会や論文集の名前
        start_patterns: dict of patterns
            Paperオブジェクトに保持するテキストの開始位置の辞書
        end_patterns: dict of pattrens
            Paperオブジェクトに保持するテキストの終了位置の辞書，Noneは最後まで
        title_position_number: int
            titleが与えられるtextboxのインデックス(ソート後)
        parse_page_numbers: list of int
            パースするページのリスト，Noneは最後まで
        paper_data_class: Paper class
            ペーパークラスのオブジェクトをストラテジーとして直接与える．
        """
        
        self.conference_name = conference_name
        
        if set(start_patterns.keys()) != set(end_patterns.keys()):
            raise ValueError("start patterns and eend patterns are not correspondding")
        
        self.title_position_number = title_position_number
        self.parse_page_numbers = parse_page_numbers  
        self.column_number = column_number
        
        self.paper_data_class = paper_data_class
        
        self.start_patterns = start_patterns
        self.end_patterns = end_patterns
        
        # パースに必要なクラスの作成
        # Layout Analysisのパラメーターを設定。縦書きの検出を有効にする。
        laparams = LAParams(detect_vertical=True)

        # 共有のリソースを管理するリソースマネージャーを作成。
        resource_manager = PDFResourceManager(caching=False)

        # ページを集めるPageAggregatorオブジェクトを作成。
        self.device = PDFPageAggregator(resource_manager, laparams=laparams)

        # Interpreterオブジェクトを作成。
        self.interpreter = PDFPageInterpreter(resource_manager, self.device)
        
        if column_number==1:
            self.SortFuncClass = SortTextbox  # クラスを変数として保持
        elif column_number==2:
            self.SortFuncClass = SortTextbox2Column
        else:
            raise ValueError("The column rather than two is not defined")
        
    def parse(self, pdf_file_path):
        """
        オーバーライドは原則禁止
        """
        self.pdf_file_name = str(pdf_file_path.stem)  # 内部メソッドからの参照用
        
        with open(pdf_file_path, "rb") as f:

            parse_text = ""
            parse_text_flag = False  # このフラッグがTrueである部分を序論とする

            for page in PDFPage.get_pages(f, pagenos=self.parse_page_numbers):
                self.interpreter.process_page(page)  # ページを処理する。
                layout = self.device.get_result()  # LTPageオブジェクトを取得。
                text_boxes = find_textboxes_recursively(layout)      

                # text_boxの座標値毎にソート，複数キーのソート
                # 少なくともこのページは全て読み込む必要があるため，非効率
                sort_func= self.SortFuncClass(layout_x0=layout.x0, layout_x1=layout.x1)
                text_boxes.sort(key=sort_func)
                
                info_dict = self.parse_info()
                paper = self.paper_data_class.parse_by_textboxes(text_boxes, info_dict)

        return paper
    
    def parse_info(self):
        """
        Paperオブジェクトによって要オーバーライド
        """
        info_dict = {}
        info_dict["conf_name"] = self.conference_name
        info_dict["pdf_name"] = self.pdf_file_name
        info_dict["start_patterns"] = self.start_patterns
        info_dict["end_patterns"] = self.end_patterns
        info_dict["title_position_number"] = self.title_position_number
        return info_dict

### テストコード 

In [32]:
start_patterns = {"序論":re.compile("[1-9]*\s*\S*(背景|はじめに|Abstract|序論|概要|Introduction)")}  # これが当てはまらないものも多い
end_patterns = {"序論":re.compile("(関連研究|提案手法|従来手法|従来研究)")}  # これが当てはまらないものも多い
#end_patterns = {"序論":None}
conference_name = "SSII2019"
title_position_number = 2
parse_page_numbers = [0]  # 正直これが一番重要(1枚目まで確認)
pdf_paper_parser = PdfParser(
                             conference_name=conference_name,
                             start_patterns=start_patterns,
                             end_patterns=end_patterns,
                             title_position_number=title_position_number,
                             parse_page_numbers=parse_page_numbers,
                            )

paper = pdf_paper_parser.parse(Path("../PDFs/IS1-20.pdf"))
print(paper.pdf_content)

{'序論': '製造・物流分野での労働力安定供給に向け，人の物理的な非定型作業を代替できるロボットの開発が求められている．さまざまな作業環境での代替実現には，作業環境の照明状況によらずに種々の物体を\n認識できる必要がある．そこで，本研究では，画像\nの局所的な鮮明度が照明条件によって変わること\nに着目し，局所鮮明度に応じて対象物の類似度\n評価方法を適切に自動選択する認識アーキテク\nチャを開発した．倉庫作業を模擬した実験系を組\nみ，撮像画像内の複数対象物の照明条件がそれ\nぞれ異なる状況下において，従来手法は認識率\nが 90%未満であるのに対し，提案手法では 98%\n以上の認識率を達成できることを確認した．'}


## カウント用のパーサ 

PdfParserの拡張はPdfParserを継承することによって行う．PdfParserクラスはPaperクラスと対のようになっており，対応するようにparse_infoに追加する．

In [33]:
class PdfParserCount(PdfParser):
    def __init__(self, count_patterns,**kwargs):
        """
        Parameters
        ----------
        count_patterns: list of pattern
            検索したいパターンのリスト
        conference_name: str
            学会や論文集の名前
        start_patterns: dict of patterns
            Paperオブジェクトに保持するテキストの開始位置の辞書
        end_patterns: dict of pattrens
            Paperオブジェクトに保持するテキストの終了位置の辞書，Noneは最後まで
        title_position_number: int
            titleが与えられるtextboxのインデックス(ソート後)
        parse_page_numbers: list of int
            パースするページのリスト，Noneは最後まで
        paper_data_class: Paper class
            ペーパークラスのオブジェクトをストラテジーとして直接与える．
        """
        kwargs["paper_data_class"] = PaperForCount()  # カウント用のPaperクラス
        super(PdfParserCount, self).__init__(**kwargs)  # 引数展開
        self.count_patterns = count_patterns
        
    def parse_info(self):
        info_dict = super(PdfParserCount, self).parse_info()
        info_dict["count_patterns"] = self.count_patterns
        
        return info_dict

### テストコード

In [34]:
start_patterns = {"序論":re.compile("[1-9]*\s*\S*(背景|はじめに|Abstract|序論|概要|Introduction)")}  # これが当てはまらないものも多い
end_patterns = {"序論":re.compile("(関連研究|提案手法|従来手法|従来研究)")}  # これが当てはまらないものも多い
count_patterns = [re.compile("ディープラーニング|深層学習"),
                  re.compile("CNN|ニューラルネットワーク"),
                  re.compile("VAE|変分オートエンコーダ"),
                  re.compile("GAN")
                 ]

#end_patterns = {"序論":None}
conference_name = "SSII2019"
title_position_number = 2
parse_page_numbers = [0]  # 正直これが一番重要(1枚目まで確認)
pdf_paper_parser = PdfParserCount(count_patterns=count_patterns,
                                  conference_name=conference_name,
                                  start_patterns=start_patterns,
                                  end_patterns=end_patterns,
                                  title_position_number=title_position_number,
                                  parse_page_numbers=parse_page_numbers,
                                  )

paper = pdf_paper_parser.parse(Path("../PDFs/IS1-10.pdf"))
pprint.pprint(paper.toDict())

{'counters': {'CNN|ニューラルネットワーク': 0,
              'GAN': 0,
              'VAE|変分オートエンコーダ': 0,
              'ディープラーニング|深層学習': 0},
 'paper_title': 'ベクタ型レーザ投影における自己位置推定のためのマーカ埋め込み手法の検討',
 'pdf_name': 'IS1-10'}


## あるディレクトリ内のpdfをパース

In [27]:
class DirectoryPdfParserJson():
    def __init__(self, 
                 dir_path,
                 pdf_parser
                ):
        
        self.dir_path = Path(dir_path)
        self.pdf_list = list(self.dir_path.glob("./*.pdf"))  # 複数回パースする必要があるため、リスト化
        
        self.pdf_parser = pdf_parser
        
    def parse(self):
        paper_list = []
        for i in tqdm(self.pdf_list):
            paper = self.pdf_parser.parse(i)
            paper_list.append(paper)
        
        return paper_list
    
    def parse_dict(self):
        
        save_dict = {self.pdf_parser.conference_name:{}}
        paper_list = self.parse()
            
        # 全てメモリで展開するので，非効率
        for paper in paper_list:
            save_dict[self.pdf_parser.conference_name][paper.pdf_name] = paper.toDict()
            
        return save_dict        

### テストコード 

### ディレクトリからのパース

In [28]:
dir_path = Path("../PDFs")
start_patterns = {"序論":re.compile("[1-9]\s*\S*(背景|はじめに|Abstract|序論|概要|Introduction|背景・目的)")}  # これが当てはまらないものも多い
end_patterns = {"序論":re.compile("[1-9]\s*\S*(関連研究|提案手法|従来手法|従来研究)")}  # これが当てはまらないものも多い
conference_name = "SSII2019"
title_position_number = 2
parse_page_numbers = [0]  # 正直これが一番重要(1枚目まで確認)
pdf_parser = PdfParser(conference_name=conference_name,
                       start_patterns=start_patterns,
                       end_patterns=end_patterns,
                       title_position_number=title_position_number,
                       parse_page_numbers=parse_page_numbers,
                       )

dir_pdf_parser = DirectoryPdfParserJson(dir_path=dir_path,
                                        pdf_parser=pdf_parser
                                       )
paper_dict = dir_pdf_parser.parse_dict()

KeyboardInterrupt: 

In [ ]:
pprint.pprint(paper_dict)

#### jsonへの保存 

In [248]:
save_path = Path("./papers.json")
with open(save_path,"w") as fw:
    json.dump(paper_dict,fw,indent=4)

### カウント

In [253]:
dir_path = Path("../PDFs")
start_patterns = {"序論":re.compile("[1-9]*\s*\S*(背景|はじめに|Abstract|序論|概要|Introduction)")}  # これが当てはまらないものも多い
end_patterns = {"序論":re.compile("(関連研究|提案手法|従来手法|従来研究)")}  # これが当てはまらないものも多い
count_patterns = [re.compile("ディープラーニング|深層学習"),
                  re.compile("CNN|ニューラルネットワーク"),
                  re.compile("VAE|変分オートエンコーダ"),
                  re.compile("GAN")
                 ]

conference_name = "SSII2019"
title_position_number = 2
parse_page_numbers = [0]  # 正直これが一番重要(1枚目まで確認)
pdf_paper_parser = PdfParserCount(count_patterns=count_patterns,
                                  conference_name=conference_name,
                                  start_patterns=start_patterns,
                                  end_patterns=end_patterns,
                                  title_position_number=title_position_number,
                                  parse_page_numbers=parse_page_numbers,
                                  )

dir_pdf_parser = DirectoryPdfParserJson(dir_path=dir_path,
                                        pdf_parser=pdf_paper_parser
                                       )
paper_list = dir_pdf_parser.parse()

#### paper_listのソート 

In [254]:
paper_list.sort(key=lambda paper_counter: tuple(paper_counter.counters.values()),reverse=True)  # OrderdDictなのでvalues順に並べればよい
print(paper_list)

[IS1-19
病理画像の semantic segmentation における
OrderedDict([(re.compile('ディープラーニング|深層学習'), 2), (re.compile('CNN|ニューラルネットワーク'), 0), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 1)]), IS1-17

OrderedDict([(re.compile('ディープラーニング|深層学習'), 2), (re.compile('CNN|ニューラルネットワーク'), 0), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)]), IS1-13
車載向け FPGA 搭載に向けたコンパクトな Residual Network による人物検知
OrderedDict([(re.compile('ディープラーニング|深層学習'), 1), (re.compile('CNN|ニューラルネットワーク'), 0), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)]), IS1-11
CNN を用いた固有画像分解による低光量画像の視認性改善
OrderedDict([(re.compile('ディープラーニング|深層学習'), 0), (re.compile('CNN|ニューラルネットワーク'), 3), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 1)]), IS1-16
Toward Person Re-identiﬁcation in Half-body Shots
OrderedDict([(re.compile('ディープラーニング|深層学習'), 0), (re.compile('CNN|ニューラルネットワーク'), 3), (re.compile('VAE|変分オートエンコーダ'), 0), (re.compile('GAN'), 0)]), IS1-03
超音波厚さ測定の効率化のためのエコー検出手法
OrderedDict([(re.compile('ディープラーニング|深層学習'), 0), 